In [1]:
import pandas as pd
import requests
import re
from shapely.geometry import Point
import geopandas as gpd

In [13]:
def acquisition_sport_centers (url_madrid_sport_centers):
    r= requests.get(url_madrid_sport_centers).json()
    df_sport_centers_raw = pd.json_normalize(r['@graph'])
    return df_sport_centers_raw

In [16]:
url_madrid_sport_centers= 'https://datos.madrid.es/egob/catalogo/212808-0-espacio-deporte.json'
df_sport_centers_raw = acquisition_sport_centers (url_madrid_sport_centers)
#df_sport_centers_raw



In [18]:
def acquisition_bicimad (path_file_csv_bicimad):
    df_bicimad_raw= pd.read_csv(path_file_csv_bicimad)
    return df_bicimad_raw

path_file_csv_bicimad= "../data/bicimad_stations.csv"
df_bicimad_raw_test = acquisition_bicimad(path_file_csv_bicimad)
#df_bicimad_raw_test

/Users/Izaskun/miniconda3/envs/bicimad/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,address.street-address,location.latitude,location.longitude,location_centros
0,Campo de Golf del Centro Nacional de Golf,CALLE ARROYO DEL MONTE 5,40.485508,-3.735279,POINT (4506826.154319164 -416104.2065551275)
1,Campo de Golf del Club de Golf Olivar de la Hi...,CALLE VIA DE DUBLIN 12,40.465351,-3.611293,POINT (4504582.255355831 -402273.7208142197)
2,Campos de Fútbol Ernesto Cotorruelo,CALLE VIA LUSITANA 5,40.381231,-3.724179,POINT (4495218.036163405 -414865.966552915)
3,Centro Comercial Dreams - Palacio Municipal de...,CALLE SILVANO 77,40.462913,-3.635734,POINT (4504310.842371287 -404999.9123872458)
4,Centro de Alto Rendimiento del Consejo Superio...,CALLE MARTIN FIERRO 5,40.436812,-3.731957,POINT (4501405.320516344 -415733.6645299961)
...,...,...,...,...,...
605,Piscina Real Canoe Madrid,CALLE PEZ VOLADOR 30,40.413487,-3.664876,POINT (4498808.844717023 -408250.5871617015)
606,Piscina SAGE Fitness,CALLE ESCALONA 59,40.397182,-3.761105,POINT (4496993.758844213 -418985.2758262118)
608,Pista de petanca Centro de Mayores La Vaguada,AVENIDA MONFORTE DE LEMOS 36,40.479033,-3.708264,POINT (4506105.338268181 -413090.5785481936)
609,Polideportivo Antonio Magariños,CALLE SERRANO 129,40.442522,-3.686096,POINT (4502040.979548661 -410617.7057179091)


In [7]:
def final_wrangling(df_bicimad_raw,df_sport_centers_raw):
    
    # Function to clean and separate values of latitude and longitude of bicimad
    def lat (x):
        b= x.split(',')
        c= b[0].replace('[','')
        d= float(c)
        return d
    def long (x):
        b= x.split(',')
        c= b[1].replace(']','')
        d= float(c)
        return d
    
    # Function to transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    def to_mercator(lat, long):
        c = gpd.GeoSeries([Point(lat, long)], crs=4326)
        c = c.to_crs(3857)
        return c
    
    # Function to return the distance in metres between two pseudo-mercator coordinates

    def distance_meters(location_centros, location_bicimad):
        return location_centros.distance(location_bicimad)
    
    #BICIMAD
    # I create the latitude and longitude columns 
    df_bicimad_raw['Latitude.bicimad'] = df_bicimad_raw.apply(lambda x: long(x['geometry_coordinates']),axis=1)
    df_bicimad_raw['Longitude.bicimad'] = df_bicimad_raw.apply(lambda x: lat(x['geometry_coordinates']),axis=1)
    
    # I create the new column location applying the function to_mercator on latitude and longitude     
    df_bicimad_raw['location_bicimad']= df_bicimad_raw.apply(lambda x: to_mercator (x['Latitude.bicimad'], x['Longitude.bicimad']), axis=1)
    
    # I keep only the columns I am interested in
    df_bicimad= df_bicimad_raw[['name', 'address', 'Latitude.bicimad', 'Longitude.bicimad', 'location_bicimad']]
    
    
    # SPORT CENTERS 
    # We first remove empty values 
    df_sport_centers_raw= df_sport_centers_raw.dropna()
    
    # I create the new column location_centros applying the function to_mercator on latitude and longitude 
    df_sport_centers_raw['location_centros']= df_sport_centers_raw.apply(lambda x: to_mercator (x['location.latitude'], x['location.longitude']), axis=1)
    
    # I keep only the columns I am interested in
    df_sport_centers= df_sport_centers_raw [['title', 'address.street-address', 'location.latitude', 'location.longitude', 'location_centros'] ]
    
    #MERGE BOTH DATAFRAMES
    
    df_merged = df_sport_centers.assign(key=0).merge(df_bicimad.assign(key=0),how='left', on = 'key')
    
    # Calculate the distance between the combination of all bicimad points and all sport center points  
    df_merged['Distance']= df_merged.apply(lambda x: distance_meters (x ['location_bicimad'], x ['location_centros']),axis=1 )
    
    # I create a table with the minimum distances grouped bye sport center
    min_distances= df_merged.groupby('title')['Distance'].min().reset_index()
    
    # I merge with the df_merged table to obtain more than the title and Distance columns
    df_final = min_distances.merge(df_merged, how='left', on = ['Distance', 'title'])
    return df_final

In [8]:
df_izas= final_wrangling (df_bicimad_raw_test, df_sport_centers_raw)

NameError: name 'df_sport_centers_raw' is not defined

In [ ]:
numero = input('I:   ')